In [ ]:
#Enabling Tensorflow using TPU engine

%tensorflow_version 1.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

TensorFlow 1.x selected.
Tensorflow version 1.15.2
Running on TPU  ['10.113.148.10:8470']
INFO:tensorflow:Initializing the TPU system: 10.113.148.10:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.113.148.10:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8551308644157903534)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11925431561414640411)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5053828486435603714)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6908679362809527296

In [ ]:
#importing necessary modules

print(tf.__version__)
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

1.15.2


In [ ]:
#Function to find semantic similarity of two sentences using Google's Universal Sentence Encoder

def semantic_similarity(cap1,cap2):
    messages=[cap1,cap2]
    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_message_encodings = embed(similarity_input_placeholder)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})    
    return cosine_similarity(message_embeddings_[0].reshape(1,-1),message_embeddings_[1].reshape(1,-1))

In [ ]:
#Mounting Drive

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Reading JSON File containing the captions generated by densecap

import json
f=open(r"drive/My Drive/Mini-Project/results.json")
data=json.load(f)
result=data["results"]
result.sort(key=lambda x:int(x["img_name"][x["img_name"].index('e')+1:x["img_name"].index('.')]))

In [ ]:
#Function to calculate euclidean distance between two points

def euclidean(x1,y1,x2,y2):
    return math.sqrt( (x1-x2)**2 + (y1-y2)**2 )

In [ ]:
#Generation of tracks

def match(frame):
    print("##############",frame["img_name"],"############")
    print("TRACKS",tracks)
    print("FINAL_TRACKS",final_tracks)
    i=0;count=0
    captions=[]
    #Choosing 5 captions with the highest weights
    while(frame["scores"][i]>=0):
        if(count<5):
            captions.append([frame["captions"][i],(frame["boxes"][i][0],frame["boxes"][i][1])])
        else:
            break
        i+=1
        count+=1
    print(captions)
    
    tracks_to_remove=[]
    for track in tracks:
        last_frame=track[2][-1]
        name=frame["img_name"]
        #Cutting threshold set to 3
        if(int(name[5:name.index('.')])-int(last_frame[5:last_frame.index('.')])>=3):
            #Removal of insignificant tracks
            if(len(track[2])<4):
                print("here")
                tracks_to_remove.append(track)
                continue
            #If the track is significant and now discountinued, gets added to the final tracklist
            else:
                final_tracks.append(track)
                tracks_to_remove.append(track)
                continue
        #Semantic Similarity threshold is set to 0.5
        else:
            captions_to_remove=[]
            print(i,track[0],"..........")
            for cap in captions:
                print(cap)
                if("<UNK>" in cap[0]):
                    captions_to_remove.append(cap)
                    continue
                x=semantic_similarity(track[0],cap[0])[0][0]
                print(x)
                #If semantic similarity threshold is satisified, eucledian distance is checked(threshold set to 125)
                if(x>=0.5):
                    if(euclidean(track[1][0],track[1][1],cap[1][0],cap[1][1])<125):
                        track[2].append(frame["img_name"])
                        print("pass")
                        captions_to_remove.append(cap)
            for rem_cap in captions_to_remove:
                captions.remove(rem_cap)
    for rem_track in tracks_to_remove:
                tracks.remove(rem_track)

    for cap in captions:
        tracks.append([cap[0],cap[1],[frame["img_name"]]])
    
    #Writing intermediate progress into file
    f=open("drive/My Drive/Mini-Project/tracks.txt",'w')
    f.write('final_tracks = '+str(final_tracks)+"\ntracks = "+str(tracks)+"\n"+frame["img_name"])
    f.close()


In [ ]:
#Reading intermediate progress from file
f=open(r'drive/My Drive/Mini-Project/tracks.txt')
c=0
for x in f:
    c+=1
    if(c==1):
        final_tracks=eval(x[15:-1])
    elif(c==2):
        tracks=eval(x[9:-1])
    else:
        index=int(x[5:x.index('.')])

In [ ]:
#Track generation frame by frame
import math
for i in range(index+1,len(result)):
  match(result[i])
print("tracks",tracks)
print("final tracks",final_tracks)

tracks [['a red circle on the sign', (611.51904296875, 208.05776977539), ['frame512 .jpg', 'frame514 .jpg', 'frame515 .jpg', 'frame516 .jpg', 'frame517 .jpg', 'frame519 .jpg']], ['a pink flower on a wall', (571.34008789062, 194.74789428711), ['frame513 .jpg', 'frame514 .jpg', 'frame515 .jpg', 'frame516 .jpg', 'frame517 .jpg', 'frame518 .jpg']]]
final tracks [['the hand of a person', (413.63818359375, 275.47778320312), ['frame0 .jpg', 'frame1 .jpg', 'frame2 .jpg', 'frame3 .jpg', 'frame5 .jpg', 'frame6 .jpg', 'frame7 .jpg', 'frame8 .jpg', 'frame9 .jpg', 'frame10 .jpg', 'frame11 .jpg', 'frame12 .jpg', 'frame13 .jpg', 'frame16 .jpg', 'frame17 .jpg', 'frame19 .jpg', 'frame20 .jpg']], ['a pink and white sign', (274.64312744141, 44.737243652344), ['frame0 .jpg', 'frame1 .jpg', 'frame1 .jpg', 'frame1 .jpg', 'frame2 .jpg', 'frame2 .jpg', 'frame2 .jpg', 'frame3 .jpg', 'frame3 .jpg', 'frame4 .jpg', 'frame6 .jpg', 'frame7 .jpg', 'frame7 .jpg', 'frame9 .jpg', 'frame9 .jpg', 'frame10 .jpg', 'frame10

In [ ]:
len(tracks)

2

In [ ]:
#Adding the last set of tracks
for i in tracks:
  final_tracks.append(i)

In [ ]:
f=open("drive/My Drive/Mini-Project/final_tracks.txt",'w')
f.write('final_tracks = '+str(final_tracks))
f.close()